# Desafio 1

Revise as avaliações da Amazon utilizando LLM e extraia as seguintes informações: resumo em até 300 caracteres em tom calmo, análise de sentimento (positivo, negativou ou neutro) e uma pontuação de -10 até +10 em relação ao sentimento detectado, sendo -10 mais negativo, 0 neutro e + 10 mais positivo.

Utilize o dataset disponibilizado neste repositório https://github.com/michelpf/dataset-customer-evaluations.

In [ ]:
!pip install openai langchain tiktoken docarray wikipedia xmltodict langchain-openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 223.4/223.4 kB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 803.6/803.6 kB 25.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 51.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.2/270.2 kB 20.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 23.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 229.5/229.5 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.8/139.8 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.1 MB/s 

In [ ]:
import os
from openai import OpenAI
import langchain
from IPython.display import display, Markdown as display_markdown
import json
import pandas as pd

In [ ]:
os.environ["OPENAI_API_KEY"] = "sua-chave"

In [ ]:
model = "gpt-3.5-turbo"

In [ ]:
from langchain.output_parsers import ResponseSchema
from langchain.output_parsers import StructuredOutputParser
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate

In [ ]:
!git clone https://github.com/michelpf/dataset-customer-evaluations

Cloning into 'dataset-customer-evaluations'...
remote: Enumerating objects: 11, done.
remote: Counting objects: 100% (11/11), done.
remote: Compressing objects: 100% (10/10), done.
remote: Total 11 (delta 1), reused 7 (delta 0), pack-reused 0
Receiving objects: 100% (11/11), 2.52 MiB | 4.86 MiB/s, done.
Resolving deltas: 100% (1/1), done.


In [ ]:
df = pd.read_csv("dataset-customer-evaluations/dataset/am_scrape_final.csv")
df.head()

,Search Query,Product Title,Link,Review
0,smartphone,Smartphone Xiaomi Note 12 4G 128GB 6GB Ram (VE...,https://www.amazon.com.br/dp/B0BZ7RJDHD,Com a necessidade de comprar um celular custo ...
1,smartphone,Smartphone Xiaomi Note 12 4G 128GB 6GB Ram (VE...,https://www.amazon.com.br/dp/B0BZ7RJDHD,Minha experiência de 10 dias de uso com o Xiao...
2,smartphone,Smartphone Xiaomi Note 12 4G 128GB 6GB Ram (VE...,https://www.amazon.com.br/dp/B0BZ7RJDHD,"Smartphone de qualidade como já esperava, boas..."
3,smartphone,Smartphone Xiaomi Note 12 4G 128GB 6GB Ram (VE...,https://www.amazon.com.br/dp/B0BZ7RJDHD,atendeu mto minhas expectativas. Antes eu usa...
4,smartphone,Smartphone Xiaomi Note 12 4G 128GB 6GB Ram (VE...,https://www.amazon.com.br/dp/B0BZ7RJDHD,"Gostei muito do celular, tem resposta rápida e..."


In [ ]:
openai_chat = ChatOpenAI(temperature=0.0, model=model)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [ ]:
sentiment_score_schema = ResponseSchema(name="score",
                             description="Pontue de -10 a +10 conforme análise sentimental de um dado texto.")
sentiment_schema = ResponseSchema(name="sentiment",
                                      description="Defina como positivo, neutro ou negativo para sintetizar\
                                      a análise sentimental do referido texto.")

response_schemas = [sentiment_score_schema, sentiment_schema]

In [ ]:
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)
format_instructions = output_parser.get_format_instructions()
format_instructions

'The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":\n\n```json\n{\n\t"score": string  // Pontue de -10 a +10 conforme análise sentimental de um dado texto.\n\t"sentiment": string  // Defina como positivo, neutro ou negativo para sintetizar                                      a análise sentimental do referido texto.\n}\n```'

In [ ]:
user_review = df.iloc[6]["Review"]
user_review

'Entrega: Comprei no dia 28 de abril e chegou 2 de mais (só não chegou antes por conta do feriado), estou com ele a 2 dias e posso afirmar que é um telefone incrívelBateria: bateria dura literalmente o dia todo, ontem sai de casa 7:30 da manhã e só fui carregar umas 00:00, isso que joguei por umas 2 horas ( call os duty e wild Rift ) e também assisti por horas, pra quem não joga dura mais de 1 dia tranquiloTela: dele é maravilhosa pra quem quer assistir, MT linda, grande e com ótimo brilho e cores, sem contar taxa de atualização de 120hz q deixa super fluídoDesemprenho: fenomenal, para quem joga e tem medo de lags e travamento, fica tranquilo, já joguei ( pubg, call of duty, wild Rift e outros ) todos rodaram perfeitamente sem nenhum travamento, mas acredito que se jogar um game extremamente pesado e com TD no máximo deve dar alguma travada, mas pra jogos básicos roda brincandoResumo: super compensa, já tive iPhone e outro xiomi ( poco x3 pro ) e msm tendo esses excelentes telefones, a

In [ ]:
review_template_sentiment = """\
Para uma revisão de clientes sobre um produto, extraia as seguintes informações:

score: Pontue de -10 a +10 conforme análise sentimental de um dado texto

sentiment: Defina como positivo, neutro ou negativo para sintetizar a
análise sentimental do referido texto

Texto da revisão do produto: {user_review}

{format_instructions}
"""

prompt = ChatPromptTemplate.from_template(template=review_template_sentiment)

customer_response_review = prompt.format_messages(user_review=user_review, format_instructions=format_instructions)

customer_response_review

[HumanMessage(content='Para uma revisão de clientes sobre um produto, extraia as seguintes informações:\n\nscore: Pontue de -10 a +10 conforme análise sentimental de um dado texto\n\nsentiment: Defina como positivo, neutro ou negativo para sintetizar a\nanálise sentimental do referido texto\n\nTexto da revisão do produto: Entrega: Comprei no dia 28 de abril e chegou 2 de mais (só não chegou antes por conta do feriado), estou com ele a 2 dias e posso afirmar que é um telefone incrívelBateria: bateria dura literalmente o dia todo, ontem sai de casa 7:30 da manhã e só fui carregar umas 00:00, isso que joguei por umas 2 horas ( call os duty e wild Rift ) e também assisti por horas, pra quem não joga dura mais de 1 dia tranquiloTela: dele é maravilhosa pra quem quer assistir, MT linda, grande e com ótimo brilho e cores, sem contar taxa de atualização de 120hz q deixa super fluídoDesemprenho: fenomenal, para quem joga e tem medo de lags e travamento, fica tranquilo, já joguei ( pubg, call of

In [ ]:
response = openai_chat.invoke(customer_response_review)
response

AIMessage(content='```json\n{\n\t"score": "9",\n\t"sentiment": "positivo"\n}\n```')

In [ ]:
result_dict = output_parser.parse(response.content)
result_dict

{'score': '9', 'sentiment': 'positivo'}